In [85]:
import os
import numpy as np
from matplotlib import pyplot as plt
from skimage.io import imread
import matplotlib.patches as patches
import pandas as pd
from tqdm import tqdm
import cv2
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from skimage import feature
import pickle
from sklearn.preprocessing import StandardScaler
import random
import pickle
from sklearn.utils import resample

In [2]:
from cars_detector.utils  import read_test_frame, read_frame
from cars_detector.sampling import sampling
from cars_detector.augmentation import random_augmentation
from cars_detector.features import compute_features, compute_colors_features
from cars_detector.makebox import non_max_suppression, box_otsu
from cars_detector.splitpictures import pyramid
from cars_detector.window_sliding import first_window_sliding, window_sliding_advanced
from cars_detector.hard_negative_sampling import hard_negative_sampling
from cars_detector.encoding import bounding_boxes_to_mask, run_length_encoding

In [116]:
boxes_array = pd.read_pickle('boxes_and_array.pkl')

In [ ]:
mask_dict = {}
for h in range(1,203):
    img = imread('./test/'+str(h).zfill(3)+'.jpg')
    boxes = boxes_array.query('image==@h').boxes.values[0]
    mask_img = np.zeros((img.shape[0],img.shape[1]))
    for box in boxes:
        mask_img[box[1]:box[1]+box[3],box[0]:box[0]+box[2]] = 1
    mask_dict[h] = mask_img

In [27]:
i = 110

img = imread('./test/'+str(i).zfill(3)+'.jpg')

search_radius = 1 # the 5th image it will be 8*5 = 40
number_of_frames_to_look = 15
boxes_to_keep = []
freq_of_presence = 0.6



boxes = boxes_array.query('image==@i').boxes.values[0]
#boxes = non_max_suppression(np.array(boxes), overlapThresh=0)

for box in boxes:
    count = 0
    number_of_remaining_frames = min(number_of_frames_to_look, 203-i)
    min_number_of_presence = np.int32(number_of_remaining_frames*freq_of_presence)
    for j in range(1,number_of_remaining_frames): # if not enough remaining images above
        future_idx = i+j
        mask_future = mask_dict[future_idx]
        y_min, y_max = max(0, box[1]-j*search_radius), min(mask_future.shape[0], box[1]+box[3]+j*search_radius)
        x_min, x_max = max(0, box[0]-j*search_radius), min(mask_future.shape[1], box[0]+box[2]+j*search_radius)
        crop = mask_future[y_min:y_max,x_min:x_max]
        if box[0] < 300 and box[1] < 300:
            continue

        
        if np.sum(crop) < 1000: # almost nothing has been found
            continue
        else:
            count += 1
        if count >= min_number_of_presence: #otherwise it is a true box
            boxes_to_keep.append(box)
            break


fig, ax = plt.subplots(figsize=(10, 8))
for x, y, dx, dy in boxes_to_keep:

    rect = patches.Rectangle((x, y), dx, dy, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

ax.imshow(img)
        


        
    